In [ ]:
import pandas as pd
from RoutePlanner.Optimisation import TravelTime
from RoutePlanner import Plot
from RoutePlanner.TemporalCellGrid import TemporalCellGrid
import pickle

In [ ]:
OptInfo = {}

# Mesh Information
OptInfo['Mesh']  = {}
OptInfo['Mesh']['Longitude Bounds (Min,Max,Width)']      = [-130+0.0001,30+0.0001,5]
OptInfo['Mesh']['Latitude Bounds (Min,Max,Width)']       = [-80,-40,2.5]

OptInfo['Mesh']['Date Range (Min,Max,dT)']               = ['2013-01-1','2013-01-30',1.0]
OptInfo['Mesh']['Homogenous Params (Threshold,Min,Max)'] = [0.25,0.05,0.95]#[0.25,0.05,0.95] [0.35,0.05,0.99]#[0.04,0.05,0.85] - Slow Vehicle, [0.12,0.05,0.85] - SDA
OptInfo['Mesh']['Current Data Path']                     = "../../Data/SOSE_surface_velocity_6yearMean_2005-2010.nc"
OptInfo['Mesh']['Ice Data Path']                         = "../../Data/bsose_i122_2013to2017_1day_SeaIceArea.nc"

# Route Information
OptInfo['Route'] = {} 
OptInfo['Route']['WayPoints']            = '../../resources/WayPoints_org.csv'
OptInfo['Route']['MaxIceExtent']         = 0.8
OptInfo['Route']['Zero Currents']        = True
OptInfo['Route']['VariableSpeed']        = False
OptInfo['Route']['Time Unit']            = 'days'

OptInfo['Route']['VehicleInfo']                = {}
OptInfo['Route']['VehicleInfo']['Speed']       = 26.5
OptInfo['Route']['VehicleInfo']['Unit']        = 'km/hr'
OptInfo['Route']['VehicleInfo']['Beam']        = 24.0
OptInfo['Route']['VehicleInfo']['HullType']    = 'slender'
OptInfo['Route']['VehicleInfo']['ForceLimit']  = 96634.5


In [ ]:
# # # Generating Cell Grid
# temporalCellGrid = TemporalCellGrid(OptInfo)
# cellGrid         = temporalCellGrid.range(OptInfo['Mesh']['Date Range (Min,Max,dT)'][0],OptInfo['Mesh']['Date Range (Min,Max,dT)'][1],j_grid=True)


# cellGrid.cellBoxes[66].landLocked  = True
# cellGrid.cellBoxes[69].landLocked  = True
# cellGrid.cellBoxes[73].landLocked  = True
# cellGrid.cellBoxes[491].landLocked = True
# cellGrid.cellBoxes[331].landLocked = True
# #cellGrid.cellBoxes[140].landLocked = True
# cellGrid.cellBoxes[396].landLocked = True
# cellGrid.cellBoxes[206].landLocked = True
# cellGrid.cellBoxes[46].landLocked = True
# cellGrid.cellBoxes[52].landLocked = True
# cellGrid.cellBoxes[65].landLocked = True

# cellGrid.iterativeSplit(0)


# with open('cellGrid.p', 'wb') as f:
#     pickle.dump(cellGrid,f)

with open('cellGrid.p', 'rb') as f:
    cellGrid = pickle.load(f)

In [ ]:
#cellGrid.plot()
map = Plot.BaseMap(logo=True,logoPos=[5,88])
map = Plot.MapMesh(cellGrid,map)
map = Plot.MapWaypoints(pd.read_csv(OptInfo['Route']['WayPoints']),map)
map = Plot.LayerControl(map,collapsed=True)
map

In [ ]:
TT = TravelTime(cellGrid,OptInfo)
RoutePaths = TT.Paths(source_waypoints=['MargueriteBay'],verbose=True,multiprocessing=False)
#RoutePaths = TT.Paths(source_waypoints=['MargueriteBay'],end_waypoints=['OrkneyPassageEntry'],verbose=True,multiprocessing=False)

In [ ]:
map = Plot.BaseMap(logo=True,logoPos=[5,88])
map = Plot.MapMesh(cellGrid,map,threshold=OptInfo['Route']['MaxIceExtent'])
map = Plot.MapJavaPaths('Java_Unsplit_2013_Unsmoothed.json',TT,map,color='blue',PathPoints=True)
map = Plot.MapPaths(RoutePaths,map,PathPoints=False)
map = Plot.MapWaypoints(pd.read_csv(OptInfo['Route']['WayPoints']),map)
map = Plot.LayerControl(map,collapsed=True)
map

In [ ]:
#SmoothedPaths = TT.PathSmoothing(maxiter=11,return_paths=True)
SmoothedPaths = TT.PathSmoothing(maxiter=300,return_paths=True)

In [ ]:
map = Plot.BaseMap(logo=True,logoPos=[5,88])
map = Plot.MapMesh(cellGrid,map,threshold=OptInfo['Route']['MaxIceExtent'])
map = Plot.MapJavaPaths('Java_Unsplit_2013_Smoothed_NoIceEffect.json',TT,map,color='blue',PathPoints=True)
map = Plot.MapPaths(SmoothedPaths,map,PathPoints=False)
map = Plot.MapWaypoints(pd.read_csv(OptInfo['Route']['WayPoints']),map)
map = Plot.LayerControl(map,collapsed=True)
map

In [ ]:
print(startIndex)
print(endIndex)

In [ ]:
TT.nc.CrossingDF

In [ ]:

RoutePaths[0]['Path']['Points'] = TT.nc.CrossingDF[['cX','cY']].to_numpy()


map = Plot.BaseMap(logo=True,logoPos=[5,88])
map = Plot.MapMesh(cellGrid,map,threshold=OptInfo['Route']['MaxIceExtent'])
map = Plot.MapWaypoints(pd.read_csv(OptInfo['Route']['WayPoints']),map)
map = Plot.MapJavaPaths('Java_Unsplit_2013_Smoothed_NoIceEffect.json',TT,map,color='blue',PathPoints=True)
map = Plot.MapPaths(RoutePaths,map,PathPoints=True)
map = Plot.LayerControl(map,collapsed=True)
map

In [ ]:
TT.nc.CrossingDF

In [ ]:
TT.nc.CrossingDF